In [3]:
import tensorflow as tf
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [4]:
data = h5py.File('training_set.hdf5', 'r')

In [7]:
X_train, Y_train, X_test, Y_test = data['game_1_x'], data['game_1_y1'], data['game_2_x'], data['game_2_y1']
X_train = np.transpose(X_train)

In [14]:
def create_placeholders(n_x, n_y):
    '''
    Arguments:
    n_x -- scalar, size of the vector = 768
    n_y -- scalar, number of classes (64)
    
    Returns:
    X -- placeholder for the data input, shape[n_x, None] and dtype tf.float32
    Y -- placeholder fo rinput labels, shape [n_y, None] and dtype tf.float32
    '''
    X = tf.placeholder(tf.float32, [n_x, None], name = 'X')
    Y = tf.placeholder(tf.float32, [n_y, None], name = 'Y')
    
    return X, Y

In [16]:
X, Y = create_placeholders(768, 64)
print ("X = " + str(X))
print ("Y = " + str(Y))

X = Tensor("X_1:0", shape=(768, ?), dtype=float32)
Y = Tensor("Y_1:0", shape=(64, ?), dtype=float32)


In [36]:
def initialize_parameters():
    '''
    Initializes parameters to build a neural network with tensorflow. The shapes are:
                        W1 : [256, 768]
                        b1 : [256, 1]
                        W2 : [128, 256]
                        b2 : [128, 1]
                        W3 : [64, 128]
                        b3 : [64, 1]
    
    Returns:
    parameters -- a dictionary of tensors containing W1, b1, W2, b2, W3, b3 
    '''
    W1 = tf.get_variable('W1', [256, 768], initializer = tf.contrib.layers.xavier_initializer())
    b1 = tf.get_variable('b1', [256, 1], initializer = tf.zeros_initializer())
    
    W2 = tf.get_variable('W2', [128, 256], initializer = tf.contrib.layers.xavier_initializer())
    b2 = tf.get_variable('b2', [128, 1], initializer = tf.zeros_initializer())
    
    W3 = tf.get_variable('W3', [64, 128], initializer = tf.contrib.layers.xavier_initializer())
    b3 = tf.get_variable('b3', [64, 1], initializer = tf.zeros_initializer())
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
    return parameters

In [37]:
tf.reset_default_graph()
with tf.Session() as sess:
    parameters = initialize_parameters()
    print('W1 = ' + str(parameters['W1']))
    print('b1 = ' + str(parameters['b1']))
    print('W2 = ' + str(parameters['W2']))
    print('b2 = ' + str(parameters['b2']))

W1 = <tf.Variable 'W1:0' shape=(256, 768) dtype=float32_ref>
b1 = <tf.Variable 'b1:0' shape=(256, 1) dtype=float32_ref>
W2 = <tf.Variable 'W2:0' shape=(128, 256) dtype=float32_ref>
b2 = <tf.Variable 'b2:0' shape=(128, 1) dtype=float32_ref>


In [38]:
def forward_propagation(X, parameters):
    '''Implements forward propagation for the model as follows;
    Linear - RELU - LINEAR - RELU - LINEAR - SOFTMAX
    Arguments:
    X -- input dataset placeholder shape (input size, number of examples)
    parametesr -- dictionary containing weights and biases 'W1', 'b1', etc.
    '''
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    Z1 = tf.add(tf.matmul(W1, X), b1)
    A1 = tf.nn.relu(Z1)
    Z2 = tf.add(tf.matmul(W2, A1), b2)
    A2 = tf.nn.relu(Z2)
    Z3 = tf.add(tf.matmul(W3, A2), b3)
    
    return Z3

In [40]:
tf.reset_default_graph()
with tf.Session() as sess:
    X, Y = create_placeholders(768, 64)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X, parameters)
    print('Z3 = ' + str(Z3))

Z3 = Tensor("Add_2:0", shape=(64, ?), dtype=float32)


In [43]:
def compute_cost(Z3, Y):
    '''compute cost
    Arguments:
    Z3 -- output of forward propagation( output of last linear unit), shape (64, examples #)
    Y -- "true" labels vector placeholder, same shape as Z3 
    returns:
    cost - tensor of cost function'''
    
    logits = tf.transpose(Z3)
    labels = tf.transpose(Y)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = labels))
    
    return cost

In [44]:
tf.reset_default_graph()
with tf.Session() as sess:
    X, Y = create_placeholders(768, 64)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X, parameters)
    cost = compute_cost(Z3, Y)
    print('cost = ' + str(cost))

cost = Tensor("Mean:0", shape=(), dtype=float32)


In [47]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001,
         num_epochs = 1500, minibatch_size = None, print_cost = True):
    
    ops.reset_default_graph() # can rerun model without overwriting tf variables
    (n_x, m) = X_train.shape # n_x: input size, m: number of examples
    n_y = Y_train.shape[0] # n_y: output size
    costs = []
    
    X, Y = create_placeholders(n_x, n_y)
    
    parameters = initialize_parameters()
    
    Z3 = forward_propagation(X, parameters)
    
    cost = compute_cost(Z3, Y)
    
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(num_epochs):
            epoch_cost = 0. #defines cost for epoch
            if minibatch_size != None:
                num_minibatches = int(m / minibatch_size)
                minibatches = random_mini_batches(X_train, Y_train, minibatch_size)
                
                for minibatch in minibatches:
                    (minibatch_X, minibatch_Y) = minibatch
                    
                    _, minibatch_cost = sess.run([optimizer, cost], feed_dict = {X : minibatch_X, Y : minibatch_Y})
                    
                    epoch_cost += minibatch_cost / num_minibatches
            else:
                _, epoch_cost = sess.run([optimizer, cost], feed_dict = {X : X_train, Y : Y_train})
            if print_cost == True and epoch % 100 == 0:
                print('Cost after epoch {0} : {1}'.format(epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
        #graph functionality
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per fives)')
        plt.title('learning rate = ' + str(learning_rate))
        plt.show()
        
        parameters = sess.run(parameters)
        print('Parameters successfully trained')
        
        correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))
        
        accuracy = tf.reduce_mean(tf.cast(correct_prediction), 'float')
        
        print('Train Accuracy:', accuracy.eval({X: X_train, Y: Y_train}))
        print('Test Accuracy:', accuracy.eval({X: X_test, Y: Y_test}))
        
        return parameters